In [1]:
# Change working directory
%cd ..

d:\HUST\OT factorization\ot-kpgf


In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import json
import pandas as pd
import numpy as np
import scipy
import os
import ot
from optimal_transport.models import KeypointFOT, FOT, LOT, EMD
from optimal_transport.experiments import Dimensionality
from typing import Tuple, Optional, List, Union, Dict

# To make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import seaborn as sns

In [3]:
n_anchors = 3
n_keypoints = 3
d = 5
k = 4
models = {
    "KeypointFOT": KeypointFOT(n_free_anchors=n_anchors, guide_mixing=0.6, stop_thr=1e-5,
                               sinkhorn_reg=0.005, temperature=0.1, div_term=1e-10),
    "FOT": FOT(n_anchors=n_anchors, sinkhorn_reg=0.1),
    "LOT": LOT(None, n_source_anchors=n_anchors, n_target_anchors=n_anchors, epsilon=100, epsilon_z=100),
    "OT": EMD(),
}

experiment = Dimensionality(model=models, exp_name="dimensionality", log_dir="logs")
experiment(hyperplane_dim=d, max_projected_dim=100, freq_projected_dim=5,
           n_components=k, cluster_samples_per_dim=5,
           n_keypoints=3, dim_noise_level=1)

Dimensions: 5, Accuracy: {'KeypointFOT': 0.25, 'FOT': 0.23, 'LOT': 0.16, 'OT': 0.17}, Runtime: 12.236928462982178s


KeyboardInterrupt: 